# Quenser Qube-Servo 2 modelling
- $\theta$ = arm angle, $\alpha$ = pendulum angle
- $\alpha$ = 0 is the upright position, while $\alpha$ = $\pi$ is the starting position

## Motor Parameters and variables

In [30]:
# pendulum parameters
mar = 0.095 #kg actual mass of arm
lar = 0.085 #m actual length of arm
mpr = 0.024 #kg actual mass of pen
lpr = 0.129 #m actual length of pen

# Motor Parameters
V_nom = 18.0       # Nominal input voltage (V)
tau_nom = 22.0/1000     # Nominal torque (mN-m) (Note: Convert to N-m)
omega_nom = 3050   # Nominal speed (RPM)
I_nom = 0.54       # Nominal current (A)
R_m = 8.4          # Terminal resistance (Ohms)
k_t = 0.042        # Torque constant (N-m/A)
k_m = 0.042        # Motor back-emf constant (V/(rad/s))
J_m = 0.000004     # Rotor inertia (kg-m^2)
L_m = 1.16e-3      # Rotor inductance (H)

## USeful functions
- Rotation matrices
- torque voltage function

In [31]:
import sympy as sym
import numpy as np
from IPython.display import display #for pretty printing
import csv

# Define the rotation matrices as functions, counterclockwise rotation
# axes system is defined similar to the one used for making the 3d plots
def rotX(th):
    return sym.Matrix([ [1.0,         0.0,          0.0],
                        [0.0, sym.cos(th), -sym.sin(th)],
                        [0.0, sym.sin(th),  sym.cos(th)]])

def rotY(th):
    return sym.Matrix([ [ sym.cos(th), 0.0, sym.sin(th)],
                        [ 0.0,         1.0,         0.0],
                        [-sym.sin(th), 0.0, sym.cos(th)]])

def rotZ(th):
    return sym.Matrix([ [sym.cos(th), -sym.sin(th), 0.0],
                        [sym.sin(th),  sym.cos(th), 0.0],
                        [0.0,    0.0,               1.0]])

def calculate_torque_voltage(i, m):
    # Step 1: Combine the motor's rotor inertia and the arm's inertia
    J_total = value(m.In[1]) + J_m
    
    # Step 2: Calculate the required torque to rotate the arm (from acceleration)
    if i > 1:
        torque = J_total * m.ddq[i, 1]
    else:
        torque = 0  # or some initial value
    
    # Step 3: Calculate the current required for the torque
    current = torque / k_t
    
    # Step 4: Calculate the motor's voltage, including the back-emf term
    if i > 1:
        voltage = current * R_m + k_m * m.dq[i, 1]
    else:
        voltage = 0  # or some initial value
    
    return torque, voltage

## Dynamic Model
- positions
- velocities
- energies
- lagrangian

In [32]:
# create symbolic variables
# system parameters
X0, Y0, Z0 = sym.symbols(['X0','Y0', 'Z0']) # fixed position of Arm
g = sym.symbols('g')

mH, Lh, InH = sym.symbols(['m_H', 'l_H', 'I_H']) # Hub
mA, La, InA = sym.symbols(['m_A', 'l_A', 'I_A']) # Arm
mP, Lp, InP = sym.symbols(['m_P', 'l_P', 'I_P']) # Pen

# generalized coordinates
th, dth, ddth       = sym.symbols(['\\theta', '\\dot{\\theta}', '\\ddot{\\theta}'])
alph, dalph, ddalph = sym.symbols(['\\alpha', '\\dot{\\alpha}', '\\ddot{\\alpha}'])

q   = sym.Matrix([  [th],  [alph]]) #group into matrices
dq  = sym.Matrix([ [dth], [dalph]])
ddq = sym.Matrix([[ddth],[ddalph]])

display(ddq) #display prints it as cool latex stuff

# Rotation Matrices
R01 = rotZ(th)
R10 = R01.T

R12 = rotX(alph)
R21 = R12.T

R02 = R12*R01
R20 = R02.T

# Positions-----------------------------------------------
##### Arm is at the origin so no need for X0 and Y0
r_Hub_1 = sym.Matrix([0, 0, 0])
r_Hub   = R10*r_Hub_1

r_Arm_1 = sym.Matrix([La*0.5, 0, 0])
r_Arm   = R10*r_Arm_1

r_Pen_2 = sym.Matrix([0, 0, Lp*0.5]) # when alpha = 0 it should be upright which is the case here
r_Pen   = r_Arm*2 + R20*r_Pen_2
display(r_Arm, r_Pen)

# Velocities-------------------------------------------------
## Linear
dr_Arm = sym.simplify(r_Arm.jacobian(q)*dq)
dr_Pen = sym.simplify(r_Pen.jacobian(q)*dq)
dr_Hub = sym.simplify(r_Hub.jacobian(q)*dq)
#display(dr_Arm,dr_Pen)

## Angular
wHub = sym.Matrix([0, 0, dth])
wArm = sym.Matrix([0, 0, dth])
wPen = R12*wArm + sym.Matrix([dalph, 0, 0])

#display(wArm, wPen)

# Energies
## Kinetic energies
Ek = lambda m, dr, w, In: (0.5*m*dr.T*dr) + (0.5*In*w.T*w)
THub = Ek(mH, dr_Hub, wHub, InH)
TArm = sym.simplify(Ek(mA, dr_Arm, wArm, InA))
TPen = sym.simplify(Ek(mP, dr_Pen, wPen, InP))
T = sym.Matrix([sym.simplify(TArm + TPen)])
#display(THub, TArm, TPen, T)

## Potential energies
VArm = 0
VHub = 0
VPen = mP*g*r_Pen[2]
V = sym.Matrix([sym.simplify(VArm + VHub + VPen)])
#display(V)

Matrix([
[\ddot{\theta}],
[\ddot{\alpha}]])

Matrix([
[ 0.5*l_A*cos(\theta)],
[-0.5*l_A*sin(\theta)],
[                   0]])

Matrix([
[ 1.0*l_A*cos(\theta) + 0.5*l_P*sin(\alpha)*sin(\theta)],
[-1.0*l_A*sin(\theta) + 0.5*l_P*sin(\alpha)*cos(\theta)],
[                                   0.5*l_P*cos(\alpha)]])

## Forces, torques and lagrangian

In [33]:
# forces & torques
tau = sym.symbols('\\tau')
vol   = sym.symbols('V')

Qtau = sym.Matrix([[tau],[0]]) # Torque tau acts in the th1 direction

Qtot = Qtau
Qall = Qtau
# ------------------------------------------------------------------------------------------------------------------
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

# put it all together
EOM = Lg1 - Lg3 + Lg4 - Qall.transpose()

EOM[1].simplify()
EOM = sym.simplify(EOM)
display(EOM)

Matrix([
[-0.5*\ddot{\alpha}*l_A*l_P*m_P*cos(\alpha) + \ddot{\theta}*(I_A + 1.0*I_P + 0.25*l_A**2*m_A + 1.0*l_A**2*m_P + 0.25*l_P**2*m_P*sin(\alpha)**2) + 0.5*\dot{\alpha}**2*l_A*l_P*m_P*sin(\alpha) + 0.5*\dot{\alpha}*\dot{\theta}*l_P**2*m_P*sin(\alpha)*cos(\alpha) - \tau],
[                                                                                           \ddot{\alpha}*(1.0*I_P + 0.25*l_P**2*m_P) - 0.5*\ddot{\theta}*l_A*l_P*m_P*cos(\alpha) - 0.25*\dot{\theta}**2*l_P**2*m_P*sin(\alpha)*cos(\alpha) - 0.5*g*l_P*m_P*sin(\alpha)]])

Matrix([
[-0.5*\ddot{\alpha}*l_A*l_P*m_P*cos(\alpha) + \ddot{\theta}*(I_A + 1.0*I_P + 0.25*l_A**2*m_A + 1.0*l_A**2*m_P + 0.25*l_P**2*m_P*sin(\alpha)**2) + 0.5*\dot{\alpha}**2*l_A*l_P*m_P*sin(\alpha) + 0.25*\dot{\alpha}*\dot{\theta}*l_P**2*m_P*sin(2*\alpha) - \tau],
[                                                                                           \ddot{\alpha}*(1.0*I_P + 0.25*l_P**2*m_P) - 0.5*\ddot{\theta}*l_A*l_P*m_P*cos(\alpha) - 0.125*\dot{\theta}**2*l_P**2*m_P*sin(2*\alpha) - 0.5*g*l_P*m_P*sin(\alpha)]])

# Pyomo stuff

In [34]:
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# Create model
m = ConcreteModel()

# Model parameters
N = 100 # number of points in the trajectory
m.N = RangeSet(N)
m.L = RangeSet(2)

# initialise some parameters
m.g   = Param(initialize = 9.81) # gravity
m.X0  = Param(initialize = 0.00) # Base position
m.Y0  = Param(initialize = 0.00)
m.Z0  = Param(initialize = 0.00)
m.h   = Param(initialize = 0.02) # timestep
m.m   = Param(m.L, initialize={1: 0.095, 2: 0.024})  # mass of links (kg)
m.len = Param(m.L, initialize={1: 0.085, 2: 0.129})  # length of links (m)


# Moment of inertia calculation
calculate_In = lambda m, l: m.m[l]*m.len[l]**2/12
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

# Variables
m.q   = Var(m.N, m.L) # position
m.dq  = Var(m.N, m.L) # velocity
m.ddq = Var(m.N, m.L) # acceleration

# define torque and voltage
m.Tc = Var(m.N) # torque


# Integration constraints---------------------------------
## positions-------------
def BwEuler_p(m,n,l): # for positions
    if n > 1:
        return m.q[n,l] == m.q[n-1,l] + m.h*m.dq[n-1,l]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to

m.integrate_p = Constraint(m.N, m.L, rule = BwEuler_p)

## velocities-------------
def BwEuler_v(m,n,l): # for velocities
    if n > 1:
        return m.dq[n,l] == m.dq[n-1,l] + m.h*m.ddq[n-1,l]
    else:
        return Constraint.Skip 

m.integrate_v = Constraint(m.N, m.L, rule = BwEuler_v)



func_map = {'sin':sin, 'cos':cos}

sym_list = [g,
            mA , mP,
            La , Lp,
            InA,InP,
            X0, Y0, Z0,
            th  ,   alph,
            dth ,  dalph,
            ddth, ddalph,
            tau]

lambEOM = {}
for i in range(1,3):
    lambEOM[i] = sym.lambdify(sym_list, EOM[i-1], modules = [func_map])

def dynamics(m, n, l):
    var_list = [m.g,
            m.m[1] , m.m[2],
            m.len[1] , m.len[2],
            m.In[1]  , m.In[2],
            m.X0, m.Y0, m.Z0,
            m.q[n, 1]   ,    m.q[n, 2],
            m.dq[n, 1]  ,   m.dq[n, 2],
            m.ddq[n, 1] ,  m.ddq[n, 2],
            m.Tc[n]]
    return lambEOM[l](*var_list) == 0
m.dynamics = Constraint(m.N, m.L, rule = dynamics)

# motor model-------------------------------------
## add motor model here


ERROR: evaluating object as numeric value: ddq[2,1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object ddq[2,1]
ERROR: Constructing component 'update_torque_voltage' from data=None failed:
        ValueError: No value for uninitialized NumericValue object ddq[2,1]


ValueError: No value for uninitialized NumericValue object ddq[2,1]

In [ ]:
# Cost function
def CostFun(m):
    torque_sum = sum(m.Tc[n]**2 for n in m.N)
    return torque_sum

m.Cost = Objective(rule = CostFun)


# Variable bounds-----------------------
AB = np.pi/2 # arm angle bounds
PB = np.pi   # pen angle bounds

bounds = {1: (-AB, AB),
          2: (-PB, PB)}

for n in range(1, N+1):
    m.Vc[n].setlb(-10)
    m.Vc[n].setub( 10)
    for link, (lb, ub) in bounds.items():
        m.q[n, link].setlb(lb)
        m.q[n, link].setub(ub)

# Conditions-------------------------------
## Initial conditions
m.q[1, 1].fix(0)
m.q[1, 2].fix(np.pi)
m.dq[1, 1].fix(0)
m.dq[1, 2].fix(0)

## final conditions
m.q[N, 1].fix(0) # final position in the middle
m.q[N, 2].fix(0) # final angle position (upright)
m.dq[N, 1].fix(0)    # final speed (0)
m.dq[N, 2].fix(0)    # final acceleration (0)

In [ ]:
# solving
opt = SolverFactory('ipopt') # standard issue, garden variety ipopt

# If you've managed to install your own version of ipopt, you can call it like:
#opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
#opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["print_level"]  = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"]     = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 300 # maximum cpu time in seconds
opt.options["Tol"]          = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
results = opt.solve(m, tee = True)

In [ ]:
# For debugging:
print(results.solver.status) # tells you if the solver had any errors/ warnings
print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

# If you want a full print-out of everything going on with your model, use:
#m.pprint() 

# This is your best friend when it comes to figuring out why something's not converging, why the solution looks weird, etc.
# It's probably most handy if you print it to a file:
#instance.pprint(filename='blah.txt')

# You can also pretty print individual elements of the model. e.g.
m.q.pprint()

# export trajectories
def write_trajectory_to_csv(m, filename, decimal_places=6):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['Time', 'Arm Angle', 'Pendulum Angle', 'Arm Velocity', 'Pendulum Velocity' , 'Torque'])
        
        for n in m.N:
            torque, voltage = calculate_torque_voltage(n, m)
            row = [
                round(m.h.value * (n-1), decimal_places),  # Time
                round(m.q[n, 1].value, decimal_places),   # Arm angle (theta)
                round(m.q[n, 2].value, decimal_places),   # Pendulum angle (alpha)
                round(m.dq[n, 1].value, decimal_places),  # Arm angular velocity
                round(m.dq[n, 2].value, decimal_places),  # Pendulum angular velocity
                round(m.Tc[n].value, decimal_places)       # torque
            ]
            writer.writerow(row)


# # Usage:
write_trajectory_to_csv(m, 'trajectory.csv', decimal_places=4)

# Animation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Create the figure and subplots for XY, XZ, YZ planes, and a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')  # 3D plot

def plot_pendulum(i, m):  
    # Clear previous plots
    ax.clear()
    
    # Set limits and titles for each axis
    lim = 0.2
    ax.set_xlim([-lim, lim])
    ax.set_ylim([-lim, lim])
    ax.set_zlim([-lim, lim])
    ax.set_xlabel('X-axis', color='red')
    ax.set_ylabel('Y-axis', color='blue')
    ax.set_zlabel('Z-axis', color='green')
    ax.grid(True)

    # Calculate positions for the pendulum
    L1bottomx = m.X0.value + m.len[1] * np.cos(m.q[i,1].value)
    L1bottomy = m.Y0.value - m.len[1] * np.sin(m.q[i,1].value)
    L1bottomz = m.Z0.value

    L2bottomx = L1bottomx + m.len[2] * np.sin(m.q[i,1].value) * np.sin(m.q[i,2].value)
    L2bottomy = L1bottomy + m.len[2] * np.sin(m.q[i,1].value) * np.cos(m.q[i,2].value)
    L2bottomz = L1bottomz + m.len[2] * np.cos(m.q[i,2].value)

    # Plot in 3D
    ax.plot([m.X0.value, L1bottomx], [m.Y0.value, L1bottomy], [m.Z0.value, L1bottomz], color='xkcd:blue')
    ax.plot([L1bottomx, L2bottomx], [L1bottomy, L2bottomy], [L1bottomz, L2bottomz], color='xkcd:red')

# Update function for the animation
update = lambda i: plot_pendulum(i, m)

# Create animation
animate = ani.FuncAnimation(fig, update, range(1,N+1), interval=50, repeat=True)

# Embed the animation in the notebook
HTML(animate.to_html5_video())